In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))


Using TensorFlow backend.


In [2]:
import numpy as np
import re
import unicodedata
from gensim.models import Word2Vec
import spacy
import nltk
import pandas as pd
from contractions import CONTRACTION_MAP #contractions.py file must be in the same directory

C:\Users\Aytu\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\Aytu\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#nlp=spacy.load('en_core_web_lg')
nlp = spacy.load('en_vectors_web_lg')

#en_vectors_web_lg.load()
total_vectors = len(nlp.vocab.vectors)
total_vectors #1070971

1070971

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [5]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text):
    text = re.sub(r'[\,]', ', ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    #text = re.sub("[^a-zA-z0-9'\s]", '', txt)
    return text

def lowercase(doc1):
    doc1=nlp(doc1)
    no_upper=' '.join([token.text.lower() for token in doc1 ])
    return no_upper

def tokenize_lemmatize(comment, join=True):
    #comment=comment.lower()
    doc=nlp(comment)
    tokens = [token for token in (doc)]
    #tokens = [token.text.lower() for token in tokens]
    #tokens=' '.join(tokens)
    #lemmas = [token.lemma_ if ((token.pos_!='PRON') & (token.lemma_.isalpha()==True)) for token in tokens]
    #lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True)& (token.is_stop==False))]
    lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True))] #& (token.is_stop==False))]
    if join:
        lemmatized=' '.join(lemmas)
        return lemmatized
    else:
        return lemmas

def tokenize_lemmatize_stops(comment, join=True):
    #comment=comment.lower()
    doc=nlp(comment)
    tokens = [token for token in (doc)]
    #tokens = [token.text.lower() for token in tokens]
    #tokens=' '.join(tokens)
    #lemmas = [token.lemma_ if ((token.pos_!='PRON') & (token.lemma_.isalpha()==True)) for token in tokens]
    #lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True)& (token.is_stop==False))]
    lemmas = [token.lemma_ for token in (tokens) if ((token.pos_!='PRON') & (token.lemma_.isalnum()==True) & (token.is_stop==False))]
    if join:
        lemmatized=' '.join(lemmas)
        return lemmatized
    else:
        return lemmas    

def remove_stops (doc):
    if type(doc) is str:
        doc=nlp(doc)
 
    return (' '.join([token.text for token in doc if token.is_stop==False]))

def remove_url (doc):
   
    return re.sub(r'http*\S+', '', doc) # remove URLs


def join_line(line):
    joined=' '.join(line)
    return joined

def transform_text(self, remove_stops=True):
    self=self.apply(lambda x: remove_url(x))
    self=self.apply(lambda x: remove_accented_chars(x))
    self=self.apply(lambda x: expand_contractions(x,CONTRACTION_MAP))
    #remove_accented_chars(text)
    self=self.apply(lambda x: remove_special_characters(x))
    #expand_contractions(text,CONTRACTION_MAP)
    self=self.apply(lambda x: lowercase(x))
    if remove_stops:
        self=self.apply(lambda x: tokenize_lemmatize_stops(x,join=True))
    else:
        self=self.apply(lambda x: tokenize_lemmatize(x,join=True))
    
    return self



In [6]:
train_val=train[125000:]
train=train[:125000]


In [7]:
test["transformed"]=transform_text(test.comment_text, remove_stops=False)

In [8]:
train_val["transformed"]=transform_text(train_val.comment_text, remove_stops=False)

In [9]:
train["transformed"]=transform_text(train.comment_text, remove_stops=False)

In [12]:
train.threat.sum()

380

In [13]:
test.to_csv('test_transformed_full.csv')
train.to_csv('train_transformed_full.csv')
train_val.to_csv('train_val_transformed_full.csv')
